In [31]:
from joblib import dump, load


In [17]:
import numpy as np 
import pandas as pd 

In [18]:
df=pd.read_csv(r"C:\Users\cl501_26\Downloads\phiusiil+phishing+url+dataset\PhiUSIIL_Phishing_URL_Dataset.csv")
df.head()

FILENAME                                 URL  URLLength  \
0  521848.txt    https://www.southbankmosaics.com         31   
1   31372.txt            https://www.uni-mainz.de         23   
2  597387.txt      https://www.voicefmradio.co.uk         29   
3  554095.txt         https://www.sfnmjournal.com         26   
4  151578.txt  https://www.rewildingargentina.org         33   

                       Domain  DomainLength  IsDomainIP  TLD  \
0    www.southbankmosaics.com            24           0  com   
1            www.uni-mainz.de            16           0   de   
2      www.voicefmradio.co.uk            22           0   uk   
3         www.sfnmjournal.com            19           0  com   
4  www.rewildingargentina.org            26           0  org   

   URLSimilarityIndex  CharContinuationRate  TLDLegitimateProb  ...  Pay  \
0               100.0              1.000000           0.522907  ...    0   
1               100.0              0.666667           0.032650  ...    0   
2               100.0              0.866667           0.028555  ...    0   
3               100.0              1.000000           0.522907  ...    1   
4               100.0              1.000000           0.079963  ...    1   

   Crypto  HasCopyrightInfo  NoOfImage  NoOfCSS  NoOfJS  NoOfSelfRef  \
0       0                 1         34       20      28          119   
1       0                 1         50        9       8           39   
2       0                 1         10        2       7           42   
3       1                 1          3       27      15           22   
4       0                 1        244       15      34           72   

   NoOfEmptyRef  NoOfExternalRef  label  
0             0              124      1  
1             0              217      1  
2             2                5      1  
3             1               31      1  
4             1               85      1  

[5 rows x 56 columns]

In [19]:
df.drop(["FILENAME","URL","URLLength","Domain","Title"],axis=1,inplace=True)

In [20]:
label=df['label']

In [21]:
label.to_csv(r'C:\Users\cl501_27\Documents\GitHub\Phishing-url-detection\Clustering\output_label.csv')

OSError: Cannot save file into a non-existent directory: 'C:\Users\cl501_27\Documents\GitHub\Phishing-url-detection\Clustering'

In [22]:
df.head()

DomainLength  IsDomainIP  TLD  URLSimilarityIndex  CharContinuationRate  \
0            24           0  com               100.0              1.000000   
1            16           0   de               100.0              0.666667   
2            22           0   uk               100.0              0.866667   
3            19           0  com               100.0              1.000000   
4            26           0  org               100.0              1.000000   

   TLDLegitimateProb  URLCharProb  TLDLength  NoOfSubDomain  HasObfuscation  \
0           0.522907     0.061933          3              1               0   
1           0.032650     0.050207          2              1               0   
2           0.028555     0.064129          2              2               0   
3           0.522907     0.057606          3              1               0   
4           0.079963     0.059441          3              1               0   

   ...  Pay  Crypto  HasCopyrightInfo  NoOfImage  NoOfCSS  NoOfJS  \
0  ...    0       0                 1         34       20      28   
1  ...    0       0                 1         50        9       8   
2  ...    0       0                 1         10        2       7   
3  ...    1       1                 1          3       27      15   
4  ...    1       0                 1        244       15      34   

   NoOfSelfRef  NoOfEmptyRef  NoOfExternalRef  label  
0          119             0              124      1  
1           39             0              217      1  
2           42             2                5      1  
3           22             1               31      1  
4           72             1               85      1  

[5 rows x 51 columns]

Detecting outliers

In [23]:
import plotly.express as px

In [24]:
"""for i in df.select_dtypes("number"):
    figure=px.box(df[i],title=f"Outlier detection for column {i}")
    figure.show()"""

'for i in df.select_dtypes("number"):\n    figure=px.box(df[i],title=f"Outlier detection for column {i}")\n    figure.show()'

## Removing outliers

In [25]:
df.columns

Index(['DomainLength', 'IsDomainIP', 'TLD', 'URLSimilarityIndex',
       'CharContinuationRate', 'TLDLegitimateProb', 'URLCharProb', 'TLDLength',
       'NoOfSubDomain', 'HasObfuscation', 'NoOfObfuscatedChar',
       'ObfuscationRatio', 'NoOfLettersInURL', 'LetterRatioInURL',
       'NoOfDegitsInURL', 'DegitRatioInURL', 'NoOfEqualsInURL',
       'NoOfQMarkInURL', 'NoOfAmpersandInURL', 'NoOfOtherSpecialCharsInURL',
       'SpacialCharRatioInURL', 'IsHTTPS', 'LineOfCode', 'LargestLineLength',
       'HasTitle', 'DomainTitleMatchScore', 'URLTitleMatchScore', 'HasFavicon',
       'Robots', 'IsResponsive', 'NoOfURLRedirect', 'NoOfSelfRedirect',
       'HasDescription', 'NoOfPopup', 'NoOfiFrame', 'HasExternalFormSubmit',
       'HasSocialNet', 'HasSubmitButton', 'HasHiddenFields',
       'HasPasswordField', 'Bank', 'Pay', 'Crypto', 'HasCopyrightInfo',
       'NoOfImage', 'NoOfCSS', 'NoOfJS', 'NoOfSelfRef', 'NoOfEmptyRef',
       'NoOfExternalRef', 'label'],
      dtype='object')

In [35]:
l1 = ['DomainLength','URLCharProb', 'TLDLength','NoOfSubDomain','NoOfObfuscatedChar','LetterRatioInURL','SpacialCharRatioInURL']
from joblib import dump, load

# Dictionary to store the outlier values for each column
outlier_info = {}

for i in l1:
  # Find outliers with IQR method

    Q1 = np.quantile(df[i],0.25)
    Q2 = np.quantile(df[i],0.50)
    Q3 = np.quantile(df[i],0.75)
    IQR  =Q3-Q1
    UB = Q3 + 1.5 *IQR
    LB = Q1 - 1.5 *IQR
    df[i] = np.where(df[i]>UB , UB , np.where(df[i]<LB,LB,df[i]))

    # Store outlier values for the column
    outlier_info[i] = {
        'Q1': Q1,
        'Q3': Q3,
        'IQR': IQR,
        'UB': UB,
        'LB': LB
    }

# Combine the DataFrame and the outlier information
data_to_save = {
    
    'outlier_info': outlier_info
}

# Save everything to a joblib file
dump(data_to_save, 'data_outliers.joblib')

print(df.head())

   DomainLength  IsDomainIP  TLD  URLSimilarityIndex  CharContinuationRate  \
0          24.0           0  com               100.0              1.000000   
1          16.0           0   de               100.0              0.666667   
2          22.0           0   uk               100.0              0.866667   
3          19.0           0  com               100.0              1.000000   
4          26.0           0  org               100.0              1.000000   

   TLDLegitimateProb  URLCharProb  TLDLength  NoOfSubDomain  HasObfuscation  \
0           0.522907     0.061933        3.0            1.0               0   
1           0.032650     0.050207        2.0            1.0               0   
2           0.028555     0.064129        2.0            1.0               0   
3           0.522907     0.057606        3.0            1.0               0   
4           0.079963     0.059441        3.0            1.0               0   

   ...  Pay  Crypto  HasCopyrightInfo  NoOfImage  NoOfCS

### One-Hot Encoding for TLD Column

In [27]:
from sklearn.preprocessing import OneHotEncoder
one = OneHotEncoder(sparse_output=False)

tld = pd.DataFrame(df["TLD"])

one.fit(tld)

TLD_encoded = one.fit_transform(tld)
# Convert the result to a DataFrame with appropriate column names
TLD_encoded_df = pd.DataFrame(TLD_encoded, columns=one.get_feature_names_out(['TLD']))

print(TLD_encoded_df)

        TLD_100  TLD_101  TLD_103  TLD_106  TLD_107  TLD_108  TLD_11  TLD_110  \
0           0.0      0.0      0.0      0.0      0.0      0.0     0.0      0.0   
1           0.0      0.0      0.0      0.0      0.0      0.0     0.0      0.0   
2           0.0      0.0      0.0      0.0      0.0      0.0     0.0      0.0   
3           0.0      0.0      0.0      0.0      0.0      0.0     0.0      0.0   
4           0.0      0.0      0.0      0.0      0.0      0.0     0.0      0.0   
...         ...      ...      ...      ...      ...      ...     ...      ...   
235790      0.0      0.0      0.0      0.0      0.0      0.0     0.0      0.0   
235791      0.0      0.0      0.0      0.0      0.0      0.0     0.0      0.0   
235792      0.0      0.0      0.0      0.0      0.0      0.0     0.0      0.0   
235793      0.0      0.0      0.0      0.0      0.0      0.0     0.0      0.0   
235794      0.0      0.0      0.0      0.0      0.0      0.0     0.0      0.0   

        TLD_111  TLD_116  .

In [ ]:
# Joblib file for encoder 

# Save the encoder
dump(one, 'encoder.joblib')

['encoder.joblib']

In [11]:
"""final_df = pd.merge(df,TLD_encoded_df,left_index=True, right_index=True)
final_df.head()"""

'final_df = pd.merge(df,TLD_encoded_df,left_index=True, right_index=True)\nfinal_df.head()'

### Removing Label from the DataSet

In [12]:
df.drop(["label","TLD"],axis=1,inplace=True)
X =df.copy()

In [13]:
print(X)

        DomainLength  IsDomainIP  URLSimilarityIndex  CharContinuationRate  \
0               24.0           0          100.000000              1.000000   
1               16.0           0          100.000000              0.666667   
2               22.0           0          100.000000              0.866667   
3               19.0           0          100.000000              1.000000   
4               26.0           0          100.000000              1.000000   
...              ...         ...                 ...                   ...   
235790          22.0           0          100.000000              1.000000   
235791          21.0           0          100.000000              0.785714   
235792          23.0           0          100.000000              1.000000   
235793          36.0           0           28.157537              0.465116   
235794          26.0           0          100.000000              1.000000   

        TLDLegitimateProb  URLCharProb  TLDLength  NoOfSubDomai

### Feature Selection 

1. Scaling 

In [14]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaled_df = pd.DataFrame(scaler.fit_transform(X),columns=X.columns)
scaled_df

DomainLength  IsDomainIP  URLSimilarityIndex  CharContinuationRate  \
0           0.479801   -0.052087            0.744382              0.713153   
1          -0.709480   -0.052087            0.744382             -0.825555   
2           0.182481   -0.052087            0.744382              0.097670   
3          -0.263499   -0.052087            0.744382              0.713153   
4           0.777121   -0.052087            0.744382              0.713153   
...              ...         ...                 ...                   ...   
235790      0.182481   -0.052087            0.744382              0.713153   
235791      0.033821   -0.052087            0.744382             -0.276016   
235792      0.331141   -0.052087            0.744382              0.713153   
235793      2.263721   -0.052087           -1.734996             -1.755936   
235794      0.777121   -0.052087            0.744382              0.713153   

        TLDLegitimateProb  URLCharProb  TLDLength  NoOfSubDomain  \
0                1.043145     0.602173   0.466430            0.0   
1               -0.905197    -0.587176  -1.410234            0.0   
2               -0.921472     0.824863  -1.410234            0.0   
3                1.043145     0.163248   0.466430            0.0   
4               -0.717171     0.349434   0.466430            0.0   
...                   ...          ...        ...            ...   
235790           1.043145     0.278151   0.466430            0.0   
235791          -0.921472    -0.219325  -1.410234            0.0   
235792          -1.021764     0.719859  -1.410234            0.0   
235793          -1.031133    -0.586822   0.466430            0.0   
235794           1.043145     0.466533   0.466430            0.0   

        HasObfuscation  NoOfObfuscatedChar  ...      Bank       Pay    Crypto  \
0            -0.045399                 0.0  ...  2.620780 -0.557340 -0.155042   
1            -0.045399                 0.0  ... -0.381566 -0.557340 -0.155042   
2            -0.045399                 0.0  ... -0.381566 -0.557340 -0.155042   
3            -0.045399                 0.0  ... -0.381566  1.794238  6.449862   
4            -0.045399                 0.0  ...  2.620780  1.794238 -0.155042   
...                ...                 ...  ...       ...       ...       ...   
235790       -0.045399                 0.0  ... -0.381566  1.794238 -0.155042   
235791       -0.045399                 0.0  ... -0.381566  1.794238 -0.155042   
235792       -0.045399                 0.0  ... -0.381566 -0.557340 -0.155042   
235793       -0.045399                 0.0  ... -0.381566 -0.557340 -0.155042   
235794       -0.045399                 0.0  ... -0.381566 -0.557340 -0.155042   

        HasCopyrightInfo  NoOfImage   NoOfCSS    NoOfJS  NoOfSelfRef  \
0                1.02681   0.099788  0.182551  0.783327     0.305222   
1                1.02681   0.301270  0.035622 -0.113046    -0.147555   
2                1.02681  -0.202435 -0.057878 -0.157865    -0.130576   
3                1.02681  -0.290583  0.276051  0.200684    -0.243770   
4                1.02681   2.744236  0.115765  1.052238     0.039216   
...                  ...        ...       ...       ...          ...   
235790           1.02681   0.313862  0.008908  0.469596     0.690083   
235791          -0.97389   0.301270 -0.071235 -0.157865     0.129771   
235792           1.02681   0.011639  0.048979  0.872964    -0.040021   
235793          -0.97389  -0.328361 -0.084592 -0.337140    -0.368284   
235794           1.02681  -0.063916 -0.004449  0.335140     1.080604   

        NoOfEmptyRef  NoOfExternalRef  
0          -0.134778         0.464130  
1          -0.134778         1.041672  
2          -0.021406        -0.274876  
3          -0.078092        -0.113413  
4          -0.078092         0.221935  
...              ...              ...  
235790     -0.021406         0.880209  
235791     -0.134778        -0.113413  
235792     -0.021406         0.110152  
235793     -0.13

In [15]:
scaled_df.columns

Index(['DomainLength', 'IsDomainIP', 'URLSimilarityIndex',
       'CharContinuationRate', 'TLDLegitimateProb', 'URLCharProb', 'TLDLength',
       'NoOfSubDomain', 'HasObfuscation', 'NoOfObfuscatedChar',
       'ObfuscationRatio', 'NoOfLettersInURL', 'LetterRatioInURL',
       'NoOfDegitsInURL', 'DegitRatioInURL', 'NoOfEqualsInURL',
       'NoOfQMarkInURL', 'NoOfAmpersandInURL', 'NoOfOtherSpecialCharsInURL',
       'SpacialCharRatioInURL', 'IsHTTPS', 'LineOfCode', 'LargestLineLength',
       'HasTitle', 'DomainTitleMatchScore', 'URLTitleMatchScore', 'HasFavicon',
       'Robots', 'IsResponsive', 'NoOfURLRedirect', 'NoOfSelfRedirect',
       'HasDescription', 'NoOfPopup', 'NoOfiFrame', 'HasExternalFormSubmit',
       'HasSocialNet', 'HasSubmitButton', 'HasHiddenFields',
       'HasPasswordField', 'Bank', 'Pay', 'Crypto', 'HasCopyrightInfo',
       'NoOfImage', 'NoOfCSS', 'NoOfJS', 'NoOfSelfRef', 'NoOfEmptyRef',
       'NoOfExternalRef'],
      dtype='object')

2. Using PCA 

In [39]:
c=scaled_df.cov() # Covariance of scaled data
eigenvalues, eigenvectors=np.linalg.eig(c)
print("Eigen values:\n",eigenvalues)
print("Eigen values shape:", eigenvalues.shape)
print("Eigen vectors shape:", eigenvectors.shape)

Eigen values:
 [8.87967467 4.02811032 2.41631387 2.01052765 1.84000879 1.76058897
 1.56648328 1.4320901  1.29646283 1.14181073 1.11805498 1.0978705
 0.03010395 0.06186552 0.06700389 0.09217365 0.15327659 1.00588667
 0.99388539 0.97805305 0.94969024 0.89726964 0.19803806 0.84947114
 0.23717309 0.80280172 0.78266669 0.75061252 0.72818847 0.71666508
 0.26433647 0.28073876 0.6741538  0.33717979 0.6449007  0.6297367
 0.59697054 0.58261488 0.55922331 0.55083301 0.50058953 0.45538965
 0.43758922 0.4291234  0.38246233 0.39104949 0.40048564 0.
 0.        ]
Eigen values shape: (49,)
Eigen vectors shape: (49, 49)


In [40]:
# Index Eigen values in descending order
idx=eigenvalues.argsort()[::-1]

#sort the eigen values in descending order
eigenvalues=eigenvalues[idx]

#sort the corresponding vectors accordingly
eigenvectors=eigenvectors[:,idx]

In [41]:
explained_var=np.cumsum(eigenvalues)/np.sum(eigenvalues)
explained_var=pd.Series(explained_var)

In [42]:
n_components=explained_var[explained_var>0.5].count()

In [43]:

n_components

42

In [44]:
from sklearn.decomposition import PCA


In [45]:
pca=PCA(n_components=n_components)
pca.fit(scaled_df)

PCA(n_components=42)

In [46]:
dump(pca,'pca.joblib')

['pca.joblib']

In [23]:
pca_df=pca.fit_transform(scaled_df)
pca_df=pd.DataFrame(pca_df,columns=['PC{}'.format(i+1)
                                    for i in range(n_components)])
pca_df

PC1       PC2       PC3       PC4       PC5       PC6       PC7  \
0      -1.355103  0.374723 -0.847538  1.444457 -0.269435  0.582533  0.518248   
1      -0.819898 -0.006518 -0.782782 -2.364139 -0.849924  0.103269  0.820576   
2      -1.217563  0.206911 -0.533419 -0.908161 -1.798203  0.532191  0.190889   
3      -2.460119  0.816660 -1.165692  0.991246  0.426025  0.335449  0.304665   
4      -3.794874  1.397170 -1.184590  0.924645  2.939978 -0.854234 -3.340179   
...          ...       ...       ...       ...       ...       ...       ...   
235790 -3.786245  0.975097  0.119499  0.975021 -0.673999  0.147243  0.596541   
235791 -1.055860  0.581192 -1.838876 -1.292704 -1.292047  0.509003 -0.100224   
235792 -2.744937  0.276077  0.315870 -0.748865 -1.738494  0.273839  0.478560   
235793  2.707944  1.164899 -2.751039  1.984488 -1.068463  0.768005 -1.107491   
235794 -2.632991  0.287991  0.718015  1.333412 -0.494270  0.190700  1.539716   

             PC8       PC9      PC10  ...      PC33      PC34      PC35  \
0       0.094391 -0.305998 -0.902024  ... -0.148379 -0.215384  0.147475   
1      -0.760786  0.270542  0.064134  ... -0.098172  0.407620 -0.119940   
2      -1.048848 -0.496761 -0.296198  ...  0.207842  0.083817 -0.129444   
3       0.919650 -0.495699 -1.303336  ... -0.220583  0.205387  0.234712   
4      -2.761792  0.205402  0.789918  ... -0.575246  0.570667 -0.391723   
...          ...       ...       ...  ...       ...       ...       ...   
235790 -0.278491  0.950621 -0.088771  ... -0.089321  0.242549 -0.288850   
235791 -0.141056 -1.192101  0.118164  ... -0.070363  0.223404 -0.967225   
235792 -1.331110 -0.087118  0.348772  ...  0.631445  0.071960 -0.725634   
235793  0.121327 -0.089341 -0.515941  ... -0.171347  0.015077 -0.097139   
235794 -1.205480  1.393336 -0.108255  ...  0.401510  0.411945 -0.859936   

            PC36      PC37      PC38      PC39      PC40      PC41      PC42  
0       0.886899  0.237549  0.741902  0.036549  0.542012 -0.363727  0.050966  
1       0.137893 -0.054673 -0.788291 -0.621070  0.541850 -0.126581  0.022341  
2       0.752606 -0.086619  1.414484  0.115978 -0.126460 -0.098412  0.042883  
3       0.234143  0.039174 -0.714097  0.018968  0.273400 -0.234085  0.047169  
4      -0.687776 -0.675614  0.519086 -0.298555 -0.460071  0.043517  0.014826  
...          ...       ...       ...       ...       ...       ...       ...  
235790  0.344621 -0.415294 -0.239474 -0.093304 -0.135130 -0.055590  0.023971  
235791 -0.707376  0.448475 -0.453751 -0.132253  0.306011 -0.211939  0.030898  
235792 -0.002972 -0.094485  0.239481 -0.003556 -0.519799  0.169899 -0.015950  
235793 -0.725029 -0.094813  0.390163 -0.119983  0.041088 -0.065019  0.049688  
235794  0.073635 -0.026764 -0.297803  0.051039 -0.237691 -0.014476  0.009120  

[235795 rows x 42 columns]

In [24]:
# To convert the pca_data and tsne_data to csv format for further model implementation

pca_df.to_csv(r'C:\Users\cl501_27\Documents\GitHub\Phishing-url-detection\Clustering\pca_data.csv',encoding='utf-8')

#### 3. Using t-SNE

In [25]:
from sklearn.manifold import TSNE


In [26]:
tsne=TSNE(n_components=2, random_state=0)
tsne_data=tsne.fit_transform(scaled_df)
tsne_data=pd.DataFrame(tsne_data)


In [27]:
tsne_data.to_csv(r'C:\Users\cl501_27\Documents\GitHub\Phishing-url-detection\Clustering\tsne_data.csv')